In [5]:
import numpy as np
import qutip as qt
from scipy.linalg import logm, expm
from qiskit.quantum_info import Operator, state_fidelity
from qiskit import QuantumCircuit
from qiskit_aer import StatevectorSimulator
from qiskit import Aer
from qiskit.circuit.library import QFT

from op_fourier_trafo import *
from boltzmann import *
from tools.classical import *
from tools.quantum import *

In [6]:
def create_U(num_qubits: int, num_energy_bits: int, hamiltonian: HamHam, 
             initial_state: np.ndarray, sigma: float) -> QuantumCircuit:
    
    qr_boltzmann = QuantumRegister(1, 'b')
    qr_energy = QuantumRegister(num_energy_bits, 'w')
    qr_sys = QuantumRegister(num_qubits, 's')
    U_circ = QuantumCircuit(qr_boltzmann, qr_energy, qr_sys)
    
    # --- Operator Fourier Transform of jump operator
    jump_op = Operator(Pauli('X'))
    oft_circ = operator_fourier_circuit(jump_op, num_qubits, num_energy_bits, hamiltonian, 
                                        initial_state=initial_state, sigma=sigma)
    U_circ.compose(oft_circ, [*list(qr_energy), *list(qr_sys)], inplace=True)
    print('OFT')

    # --- Act on Boltzmann coin
    boltzmann_circ = lookup_table_boltzmann(num_energy_bits)
    U_circ.compose(boltzmann_circ, [qr_boltzmann[0], *list(qr_energy)], inplace=True)
    print('Boltzmann')
    
    return U_circ

In [ ]:
def create_U_dagger(num_qubits: int, num_energy_bits: int, hamiltonian: HamHam, 
                    initial_state: np.ndarray, sigma: float) -> QuantumCircuit:
    
    qr_boltzmann = QuantumRegister(1, 'b')
    qr_energy = QuantumRegister(num_energy_bits, 'w')
    qr_sys = QuantumRegister(num_qubits, 's')
    U_dagger_circ = QuantumCircuit(qr_boltzmann, qr_energy, qr_sys)
    
    # --- Reverse act on Boltzmann coin
    boltzmann_circ = lookup_table_boltzmann(num_energy_bits)

In [7]:
np.random.seed(667)
num_qubits = 3
num_energy_bits = 6
bohr_bound = 2 ** (-num_energy_bits + 1) #!
eps = 0.05
sigma = 10
eig_index = 2
T = 1
delta = 0.1
shots = 1000

hamiltonian = find_ideal_heisenberg(num_qubits, bohr_bound, eps, signed=False, for_oft=True)
rescaled_coeff = hamiltonian.rescaled_coeffs
# Corresponding Trotter step circuit
trotter_step_circ = trotter_step_heisenberg(num_qubits, coeffs=rescaled_coeff, symbreak=True)
hamiltonian.trotter_step_circ = trotter_step_circ

#* Initial state = eigenstate
initial_state = hamiltonian.eigenstates[:, eig_index]
initial_state = Statevector(initial_state)
print(f'Initial energy: {hamiltonian.spectrum[eig_index]}')

Original spectrum:  [-3.5    -2.9921 -2.3725 -1.5     1.6025  2.1     2.8895  3.7725]
Ideal spectrum:  [-0.0071  0.0255  0.0653  0.1214  0.3209  0.3529  0.4036  0.4604]
Nonrescaled coefficients:  [0.9 0.7 0.9 1. ]
Rescaled coefficients:  [0.05785714 0.045      0.05785714 0.06428571]
Initial energy: 0.06534196648337862


In [10]:
qr_boltzmann = QuantumRegister(1, 'b')
qr_energy = QuantumRegister(num_energy_bits, 'w')
qr_sys = QuantumRegister(num_qubits, 's')
circ = QuantumCircuit(qr_boltzmann, qr_energy, qr_sys)
U = create_U(num_qubits, num_energy_bits, hamiltonian, initial_state, 0.1)

Energy before jump: 0.06534196648337864
Jump applied to 1th qubit
Energy after jump: 0.3066008839869134
Energy jump: 0.24125891750353473
OFT
Boltzmann
